首先恭喜各位，通过第一轮的简历筛选，进入线上测试环节。

通过这次线上测试，希望大家能够在限定的时间内，尽自己最大努力，完成一些数据处理和机器学习建模任务，展现自己对机器学习建模工作的熟悉程度。

## 任务说明
请在这个jupyter notebook中
* 补充完成任务所需要的python代码。可以直接在code cell中编写代码；或者将代码写在.py文件中后在这个notebook中import使用
* 利用markdown cell回答相关的问题或者解释分析思路

完成后，在自己的github上建立一个repo。将这个notebook，相关的.py文件，保存处理结果的csv文件等，push到建立的repo。将repo链接提交给面试官。


## 注意事项：
1. 使用python3.6以上版本，如果使用```pandas```等python库，请先升级到最新版本。
2. 请确保面试官在clone各位发布的repo后，能够跑通这个notebook中的代码。
3. 除了实现任务所需的功能，编写的代码具有高可维护性，也是我们非常希望候选人具备的能力。请使用可读性强的命名，避免代码冗余，以及体现其他良好的编程实践。（可参考Refactoring by Kent Beck and Martin Fowler一书中关于"bad code smell"的讨论）
4. 这个测试的目的是检验候选人是否具备高效的完成建模相关工作的能力，对相关工具的熟悉程度也是考核的目标之一。不鼓励重复造轮子。建议充分利用```pandas```, ```sklearn```等已有python库完成任务。
5. 请展现自己真实的能力，不要请人代劳。如果候选人试用期阶段表现的实际工作能力明显低于线上测试表现的能力，候选人会被认为采用非诚信手段通过线上测试，将予以坚决辞退。
6. 职场中，Deadline是非常严肃的。无论完成度如何，请务必在截止时间前，提交自己的结果。逾期提交将不予受理。

## 环境依赖列表


```
python=3.6
scikit-learn=0.19.0
pandas=0.23.4
numpy=1.13.1
xgboost=0.90
openpyxl=2.6.2

```

## 机器学习建模（任务1～任务2）
以下任务基于虚拟的建模数据dataset.csv

建模数据集构成
* id：样本的编号
* dataset：划分了哪些数据用于train, 那些用于test
* label:  0或者1, 只有train数据才给出了label
* 其实字母f开头的列为建模特征

### 任务1

请根据train数据，使用3种指定的机器学习算法分别训练模型。这3种算法是
* xgboost
* logistic regression
* random forest

要求:
* 预测目标为label列
* 模型优化的目标为auc_roc
* 需要对特征进行筛选。每种算法训练的模型，最后使用的特征控制在20个以内。
* 每种算法得到的模型，将模型在test数据集的预测结果(0~1之间的一个分值)保存在prediciton.csv文件中。在候选人提交结果后，面试官会计算候选人的模型在test数据集的auc_roc。 csv应该包含id, 以及每个模型对应的proba。请参考作为样例的prediciton_sample.csv中的形式

### 简述建模思路

#### 数据加载及预处理

从文件load数据, 由于有大量的缺失数据, 先对缺失进行补齐(这里选择的补齐策略是固定补-1), 之后对数据进行切分, 分成训练集, 验证集和测试集. 

#### 基于不同策略(算法建模)

三个算法的实现过程类似, 以xgboost为例简略说明过程, 详细请参考`tast_one.py`里的方法`build_model`部分.


##### 特征选择(Model based)

特征选择的方法有很多, 这里选择的是基于模型的特征选择, 就是在训练集(全属性)上训练一个模型, 得到属性的重要度, 然后启发式的寻找哪些组合最好

1. 基于训练数据(这里用的是所有属性), 构建xgboost模型.
2. 抽取属性重要程度(一个分值列表, 对应每个属性的重要程度, 并按大到小排序)
3. 循环每个值作为阈值来做属性选择
  
  a. 找到对应属性组合
  
  b. 基于属性组合转换训练数据, 并生成新的训练模型
  
  c. 在验证集上计算模型的auc值
 
4. 基于上个循环的到每种组合的auc列表, 计算移动平均值(减小偶然因素), 获取auc最大的组合对应的阈值
5. 基于找到的阈值, 选择出来对应的属性组合作为结果


##### 构建最终模型

用选择后的属性组合, 转换数据后, 生成最终的xgboost模型

##### 生成预测结果

用生成的模型在test数据上做预测, 并保留预测结果


In [1]:
# common import
import numpy as np
import pandas as pd

In [2]:
# 引入文件
import task_one

In [3]:
# 开启debug级别的日志
import logging
log = task_one.init_log(logging.DEBUG)

In [6]:
# 加载数据
train_data_X, val_data_X, train_data_y, val_data_y, test_data = task_one.load_data()

In [7]:
# 构建xgboost模型

# 属性选择的范围如果扩展到60(最后一个参数控制, 默认20), 效果可能好些, 试验中, 在其他参数不变, 这里在47个属性时候, 验证集上的推广性能最好
#xgboost_pred_result_df = task_one.build_model(train_data_X, val_data_X, train_data_y, val_data_y, test_data, task_one.init_XGBClassifier, 60)

xgboost_pred_result_df = task_one.build_model(train_data_X, val_data_X, train_data_y, val_data_y, test_data, task_one.init_XGBClassifier)

2019-05-30 16:17:58,740-INFO - build model
2019-05-30 16:18:07,384-INFO - feature select
2019-05-30 16:18:07,623-DEBUG - auc for top 1 features, score on train is:0.748813, score on val is: 0.792492
2019-05-30 16:18:07,830-DEBUG - auc for top 2 features, score on train is:0.859110, score on val is: 0.880941
2019-05-30 16:18:08,028-DEBUG - auc for top 3 features, score on train is:0.861332, score on val is: 0.881063
2019-05-30 16:18:08,235-DEBUG - auc for top 4 features, score on train is:0.867144, score on val is: 0.893268
2019-05-30 16:18:08,436-DEBUG - auc for top 5 features, score on train is:0.880277, score on val is: 0.890314
2019-05-30 16:18:08,626-DEBUG - auc for top 6 features, score on train is:0.881479, score on val is: 0.892541
2019-05-30 16:18:08,853-DEBUG - auc for top 7 features, score on train is:0.882744, score on val is: 0.893139
2019-05-30 16:18:09,096-DEBUG - auc for top 8 features, score on train is:0.908178, score on val is: 0.919450
2019-05-30 16:18:09,336-DEBUG -

In [8]:
# 构建LR模型
# 属性选择的范围如果扩展到60(最后一个参数控制, 默认20), 效果可能好些, 试验中, 在其他参数不变, 这里在25个属性时候, 验证集上的推广性能最好
#lr_pred_result_df = task_one.build_model(train_data_X, val_data_X, train_data_y, val_data_y, test_data, task_one.init_LogisticRegression, 60)

lr_pred_result_df = task_one.build_model(train_data_X, val_data_X, train_data_y, val_data_y, test_data, task_one.init_LogisticRegression)

2019-05-30 16:18:26,261-INFO - build model
2019-05-30 16:18:36,992-INFO - feature select
2019-05-30 16:18:37,084-DEBUG - auc for top 1 features, score on train is:0.686796, score on val is: 0.658512
2019-05-30 16:18:37,172-DEBUG - auc for top 2 features, score on train is:0.741894, score on val is: 0.773374
2019-05-30 16:18:37,247-DEBUG - auc for top 3 features, score on train is:0.743281, score on val is: 0.779649
2019-05-30 16:18:37,327-DEBUG - auc for top 4 features, score on train is:0.746530, score on val is: 0.779704
2019-05-30 16:18:37,401-DEBUG - auc for top 5 features, score on train is:0.751181, score on val is: 0.784023
2019-05-30 16:18:37,481-DEBUG - auc for top 6 features, score on train is:0.768641, score on val is: 0.782570
2019-05-30 16:18:37,565-DEBUG - auc for top 7 features, score on train is:0.793621, score on val is: 0.799861
2019-05-30 16:18:37,705-DEBUG - auc for top 8 features, score on train is:0.799239, score on val is: 0.800880
2019-05-30 16:18:37,856-DEBUG -

In [9]:
# 构建RF模型
# 属性选择的范围如果扩展到60(最后一个参数控制, 默认20), 效果可能好些, 试验中, 在其他参数不变, 这里在24个属性时候, 验证集上的推广性能最好
#rf_pred_result_df = task_one.build_model(train_data_X, val_data_X, train_data_y, val_data_y, test_data, task_one.init_RandomForestClassifier, 60)

rf_pred_result_df = task_one.build_model(train_data_X, val_data_X, train_data_y, val_data_y, test_data, task_one.init_RandomForestClassifier)

2019-05-30 16:18:46,623-INFO - build model
2019-05-30 16:18:47,574-INFO - feature select
2019-05-30 16:18:47,905-DEBUG - auc for top 1 features, score on train is:0.815111, score on val is: 0.685815
2019-05-30 16:18:48,194-DEBUG - auc for top 2 features, score on train is:0.994804, score on val is: 0.831144
2019-05-30 16:18:48,488-DEBUG - auc for top 3 features, score on train is:0.995937, score on val is: 0.887672
2019-05-30 16:18:48,802-DEBUG - auc for top 4 features, score on train is:0.999944, score on val is: 0.898722
2019-05-30 16:18:49,175-DEBUG - auc for top 5 features, score on train is:0.999941, score on val is: 0.887155
2019-05-30 16:18:49,481-DEBUG - auc for top 6 features, score on train is:0.999938, score on val is: 0.909466
2019-05-30 16:18:49,817-DEBUG - auc for top 7 features, score on train is:0.999972, score on val is: 0.910485
2019-05-30 16:18:50,135-DEBUG - auc for top 8 features, score on train is:0.999963, score on val is: 0.905907
2019-05-30 16:18:50,525-DEBUG -

In [10]:
# 整理结果并输出
import pandas as pd
final_result_df = pd.DataFrame({'lr': lr_pred_result_df['pred']
                                ,'xgboost': xgboost_pred_result_df['pred']
                                , 'rf': rf_pred_result_df['pred']},
                               index=xgboost_pred_result_df.index,
                               columns=['lr', 'xgboost', 'rf'])
final_result_df.index.name = 'id'
log.info('final result: %s', final_result_df.to_string())
final_result_df.to_csv('prediction.csv', sep='\t')

2019-05-30 16:19:02,956-INFO - final result:             lr   xgboost        rf
id                                
3000  0.050920  0.040566  0.000000
3001  0.265287  0.067932  0.033333
3002  0.214462  0.067932  0.166667
3003  0.336578  0.038070  0.033333
3004  0.339464  0.038070  0.033333
3005  0.098979  0.055302  0.133333
3006  0.065869  0.067932  0.066667
3007  0.109974  0.067932  0.000000
3008  0.086946  0.038070  0.000000
3009  0.086734  0.038070  0.000000
3010  0.048565  0.038070  0.200000
3011  0.047973  0.038070  0.166667
3012  0.039811  0.038070  0.166667
3013  0.018508  0.040566  0.000000
3014  0.010774  0.040566  0.033333
3015  0.018253  0.040566  0.000000
3016  0.040413  0.038070  0.000000
3017  0.040413  0.038070  0.000000
3018  0.022425  0.038070  0.000000
3019  0.032046  0.067932  0.000000
3020  0.032246  0.067932  0.000000
3021  0.170424  0.067932  0.033333
3022  0.060151  0.038070  0.000000
3023  0.059715  0.038070  0.000000
3024  0.278801  0.067932  0.000000
3025  0.26

### 任务2
如果需要在产品中使用。你会推荐使用之前哪个算法得到的模型？解释为什么。

这个要看业务方面需要什么类型的模型, 比如构建速度是否有要求, 预测速度是否有要求, 分类精度以及分错的代价有多大等, 这里因为没有上下文, 其他方面也没有恶化到不能接受(比如太慢等), 这边就以在验证集上的auc为标准, 选择auc最大的. 这里如果把属性个数提高到60我会选择随机森林, 它的推广auc在0.943(对应的xgboost的在0.93, lr的在0.86).

### 任务3

任务1～任务4体现了从得到建模数据，到完成建模实验的环节。

然而，对于整个建模项目来说，以上环节只是一部分工作内容。请候选人根据自己的经验和理解，描述从接到业务部门的需求开始（假设是一个0，1分类的监督式机器学习任务），到监控模型在产品中的使用情况为止，有哪些重要的工作环节？在这些过程中，机器学习建模工程师和业务部门成员如何分工？如何合作？

建议通过流程图配合必要的文字描述，表达自己的理解。





### 对建模项目主要环节的理解

#### 1. 需求定义和整理阶段

这个阶段非常关键, 方向不对, 努力白费, AI部门需要跟业务部门一起确定需求的内容, 明确我们要达到的目标, 特别是识别非功能性方面的需求, 比如性能方面, 可用性方面, 安全方面等, 另外针对业务领域方面要注意一些这个领域特有的特点(往往对业务来说, 他们认为是常识, 可能就不会说这块), 比如: 对病人是否得某种病的识别, 这里有两种错误, 把有病的判成没病的, 把没病的判成有病的, 这两种错误可能代价差别很大, 需要有针对性的考虑.

这个阶段可能会反复个几次, 多次确认, 尽量在早期阶段就把这块给确定下来.

#### 2. 识别外部依赖

这个环节是识别要完成这个项目, 需要哪些兄弟部门的协作, 比如数据从哪来, 以什么形式获得数据, IT资源的申请等, 识别出来外部依赖之后, 特别是对其中不确定的部分, 尽早确定, 避免block后续的开发上线等. 

#### 3. 任务细分

团队里有各种的成员, 特别是团队比较大(10个人以上)的时候, 基本上不太可能每个人对所有的事情都熟悉, 或者把全部事情跟所有人都说清楚, 也没那个必要, 需要对把需求进行细分, 这样团队里具体的成员只需要关注在跟自己相关的task上, 更明确也更高效.

#### 4. 数据特点分析

Garbage In Garbage Out, 想要达到我们的目标(二值分类), 我们必须对数据本身有清晰的认识, 发现数据的特点, 以及结合给定领域下, 这些特点的意义(而不是仅仅停留在这一列最大值是啥, 均值是啥), 对数据本身特点弄明白了, 还要明确我们的目的, 比如这里的二值分类, 代表的什么意义, 以及两种错误代表的意义(两种错误是指, 把0看成1, 和把1看成0, 业务领域这两种错误往往差别很大). 

#### 5. 算法相关

现在存在的算法非常多, 基本我们碰到的问题绝大部分都能找到现成的相关算法. 所以这个阶段不是研制算法, 更多的是从现有算法里挑选合适的算法. 

##### 5.1 算法选择以及原型实现

考虑问题的特点, 以及问题的类别初步挑选算法, 比如目的是分类还是聚类, 是数值型的数据还是离散型的, 有没有缺失, 支不支持并行等. 挑选算法之后, 就是基于现有的lib包, 做一些原型出来, 以及初步的调优(这里不会很深入).

##### 5.2 模型对比分析, 选择最终模型

在原型的基础上, 对模型进行对比分析, 比如精度, 两种错误率, auc, 模型复杂度(构建的复杂度和模型出来后做预测的复杂度), 综合考虑, 选择最终模型, 并做深入的优化.  

#### 6. 非算法部分

一个AI项目里, AI部分可能占的比例都不到30%, 更多的是业务相关代码, 以及各种adapter, 如: API接口, 各种ETL转换等.


#### 7. 内部测试

测试是保证质量的关键部分.

完成建模以及开发后, 内部要做充分的测试, 特别是对医疗领域, 出错的代价可能很大.

#### 8. 业务部门UAT测试

最终交付客户使用之前(或者业务部门之前), 需要业务部门做最终的UAT(验收测试), 确定做的东西是他们想要的, 当然这里的UAT可能不会是到最后才做, 可能是细分成几个小阶段, 每个阶段完成给他们UAT一部分, 小步慢走, 保证最终的结果是他们想要的.

#### 9. 上线及运维

UAT之后是具体的上线, 以及上线之后的配套的监控, 及时发现线上问题, 并做好相应的应对准备.

#### 10. 变更管理

这个是贯穿始终的环节, 项目的整个生命周期里, 大部分会和最初的时候有区别, 要跟业务部门协调好这方面的事情, 不可能不改, 也不可能随意改. 这个灵活性很大, 只能结合实际情况来分析和实践了.


## 数据处理

### 任务4

history_df中存放了每个id，在不同time，对应的A, B的数值。

In [11]:
history_data = pd.read_csv('history_data.csv', parse_dates=['time'])
history_data


,id,time,A,B
0,1,2019-01-26 08:52:00,3.797676,8.432960
1,1,2019-01-29 14:59:00,NaN,4.157219
2,1,2019-01-13 00:01:00,6.516694,4.922487
3,1,2019-02-01 00:27:00,2.627256,NaN
4,1,2019-03-26 00:32:00,8.939391,NaN
5,2,2019-01-01 00:14:00,NaN,6.449302
6,2,2019-01-26 00:52:00,3.629338,1.529292
7,2,2019-02-04 00:05:00,4.299086,NaN
8,2,2019-02-28 00:51:00,NaN,3.140770
9,2,2019-03-01 00:44:00,6.353059,5.926424


sample_df存放了一些id和obs_time的组合。

In [12]:
sample_df = pd.read_csv('sample.csv',  parse_dates=['obs_time'])
sample_df

,id,obs_time
0,1,2019-03-07
1,1,2019-02-01
2,2,2019-03-15
3,2,2019-02-01


对sample_df中每个id和obs_time的组合，计算离obs_time过去10, 60天内， 该id对应的A, B的最大值, 非空记录数量。将结果存在不同的列中。

例如:


最终的结果包含的列:
* id
* obs_time
* 10天内A平均值
* 10天内A非空样本数量
* 60天内A平均值
* 60天内A非空样本数量
* 10天内B平均值
* 10天内B非空样本数量
* 60天内B平均值
* 60天内B非空样本数量

将计算结果存为excel保存

In [14]:
# 两个df基于id做join
raw_data_df = pd.merge(sample_df, history_data, on='id')

# 添加辅助列, diff, 值是obs_time减去time
raw_data_df['diff'] = raw_data_df['obs_time'] - raw_data_df['time']

# 差值在10天内的有效记录
raw_data_df_ten_days = raw_data_df[(raw_data_df['diff'] > pd.Timedelta('0 day')) & (raw_data_df['diff'] < pd.Timedelta('10 days'))]
print('\n\n差值在10天内的有效记录')
print(raw_data_df_ten_days.to_string())
# 聚合得到分组后A, B的均值和非空计数
ten_days_agg_result_pd = raw_data_df_ten_days.groupby(['id', 'obs_time']).agg({'A': ['mean', 'count'], 'B': ['mean', 'count']})
print('\n差值在10天内的计算结果')
print(ten_days_agg_result_pd)

# 差值在60天的有效记录
raw_data_df_sixty_days = raw_data_df[(raw_data_df['diff'] > pd.Timedelta('0 day')) & (raw_data_df['diff'] < pd.Timedelta('60 days'))]
print('\n\n差值在60天内的有效记录')
print(raw_data_df_sixty_days.to_string())

# 聚合得到分组后A, B的均值和非空计数
sixty_days_agg_result_pd = raw_data_df_sixty_days.groupby(['id', 'obs_time']).agg({'A': ['mean', 'count'], 'B': ['mean', 'count']})
print('\n差值在60天内的计算结果')
print(sixty_days_agg_result_pd)

# 合并最终结果
merged_result = pd.merge(ten_days_agg_result_pd, sixty_days_agg_result_pd, on=['id', 'obs_time'], how='outer')
merged_result.index.name = ['id','obs_time']
merged_result.columns = ['10天内A平均值','10天内A非空样本数量','60天内A平均值','60天内A非空样本数量','10天内B平均值','10天内B非空样本数量','60天内B平均值','60天内B非空样本数量']
# print(merged_result)

# 输出到Excel
merged_result.to_excel('calculate_result.xlsx')
print('\n\n输出到结果到calculate_result.xlsx完成')



差值在10天内的有效记录
    id   obs_time                time         A         B            diff
5    1 2019-02-01 2019-01-26 08:52:00  3.797676  8.432960 5 days 15:08:00
6    1 2019-02-01 2019-01-29 14:59:00       NaN  4.157219 2 days 09:01:00
16   2 2019-02-01 2019-01-26 00:52:00  3.629338  1.529292 5 days 23:08:00

差值在10天内的计算结果
                      A               B      
                   mean count      mean count
id obs_time                                  
1  2019-02-01  3.797676     1  6.295089     2
2  2019-02-01  3.629338     1  1.529292     1


差值在60天内的有效记录
    id   obs_time                time         A         B             diff
0    1 2019-03-07 2019-01-26 08:52:00  3.797676  8.432960 39 days 15:08:00
1    1 2019-03-07 2019-01-29 14:59:00       NaN  4.157219 36 days 09:01:00
2    1 2019-03-07 2019-01-13 00:01:00  6.516694  4.922487 52 days 23:59:00
3    1 2019-03-07 2019-02-01 00:27:00  2.627256       NaN 33 days 23:33:00
5    1 2019-02-01 2019-01-26 08:52:00  3.797676  8.4329